In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def unpickle(our_file):
  import pickle
  with open(our_file, 'rb') as fo:
    data = pickle.load(fo, encoding='bytes')
  return data

In [3]:
train_1=unpickle("data_batch_1")
train_2=unpickle("data_batch_2")
train_3=unpickle("data_batch_3")
test_data=unpickle('test_batch')

In [4]:
x_train= np.concatenate([train_1[b'data'], train_2[b'data'], train_3[b'data']])
x_train= x_train.reshape(-1,3,32,32).transpose(0,2,3,1)
x_train= x_train.astype(float)/255

In [5]:
y_train=np.concatenate([train_1[b'labels'], train_2[b'labels'], train_3[b'labels']])
y_train

array([6, 9, 9, ..., 3, 7, 1])

In [6]:
x_test=test_data[b'data'].reshape(-1,3,32,32).transpose(0,2,3,1)
y_test=np.array(test_data[b'labels'])
x_test=x_test.astype(float)/255

In [7]:
x_add_test=np.load('test_additional.npy')
y_add_test=np.load('labels.npy')
x_add_test=x_add_test.astype(float)/255
x_add_test[:1]

array([[[[0.63137255, 0.45882353, 0.10196078],
         [0.66666667, 0.57254902, 0.14509804],
         [0.63921569, 0.30196078, 0.18039216],
         ...,
         [0.5372549 , 0.36862745, 0.15686275],
         [0.51764706, 0.23921569, 0.16862745],
         [0.39607843, 0.35294118, 0.20784314]],

        [[0.45882353, 0.41568627, 0.07058824],
         [0.58431373, 0.34901961, 0.30196078],
         [0.75686275, 0.49803922, 0.2       ],
         ...,
         [0.49803922, 0.39215686, 0.        ],
         [0.52941176, 0.20784314, 0.07058824],
         [0.35294118, 0.28627451, 0.11764706]],

        [[0.54117647, 0.42745098, 0.29411765],
         [0.44313725, 0.39215686, 0.12941176],
         [0.77254902, 0.4       , 0.22745098],
         ...,
         [0.5254902 , 0.28235294, 0.17254902],
         [0.4745098 , 0.47058824, 0.03921569],
         [0.64313725, 0.28627451, 0.08235294]],

        ...,

        [[0.04313725, 0.60784314, 0.75294118],
         [0.26666667, 0.50980392, 0.64705882]

# Part 1 (Using Sigmoid)

In [8]:
class using_sigmoid():
  def __init__(self):
    cnn = models.Sequential()
    self.cnn=cnn
  def train_cnn_using_sigmoid(self,x_train, y_train, epochs=15):
    self.cnn.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='sigmoid', input_shape=(32, 32, 3)))
    self.cnn.add(layers.MaxPooling2D((2, 2)))
    self.cnn.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='sigmoid'))
    self.cnn.add(layers.MaxPooling2D((2, 2)))


    self.cnn.add(layers.Flatten())
    self.cnn.add(layers.Dense(64, activation='sigmoid'))
    self.cnn.add(layers.Dense(10, activation='softmax'))

    #Compile our models
    self.cnn.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    #fit data our cnn models
    return self.cnn.fit(x_train, y_train, epochs=15)
  def test_cnn_using_sigmoid(self, x_test,y_test):
    return self.cnn.evaluate(x_test,y_test)
  def predict_using_sigmoid(self,x_test):
    return self.cnn.predict(x_test)

In [9]:
code1=using_sigmoid()
code1.train_cnn_using_sigmoid(x_train, y_train)

Epoch 1/15
938/938 [==============================] - 13s 7ms/step - loss: 2.1056 - accuracy: 0.2285
Epoch 2/15
938/938 [==============================] - 3s 4ms/step - loss: 1.8714 - accuracy: 0.3261
Epoch 3/15
938/938 [==============================] - 3s 4ms/step - loss: 1.6984 - accuracy: 0.3884
Epoch 4/15
938/938 [==============================] - 4s 5ms/step - loss: 1.5784 - accuracy: 0.4358
Epoch 5/15
938/938 [==============================] - 3s 4ms/step - loss: 1.4901 - accuracy: 0.4637
Epoch 6/15
938/938 [==============================] - 3s 4ms/step - loss: 1.4222 - accuracy: 0.4903
Epoch 7/15
938/938 [==============================] - 3s 4ms/step - loss: 1.3728 - accuracy: 0.5068
Epoch 8/15
938/938 [==============================] - 4s 4ms/step - loss: 1.3329 - accuracy: 0.5222
Epoch 9/15
938/938 [==============================] - 3s 4ms/step - loss: 1.2946 - accuracy: 0.5373
Epoch 10/15
938/938 [==============================] - 3s 4ms/step - loss: 1.2565 - accuracy: 0.552

In [10]:
code1.test_cnn_using_sigmoid(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2481 - accuracy: 0.5560


[1.248109221458435, 0.5559999942779541]

In [11]:
code1.test_cnn_using_sigmoid(x_add_test,y_add_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2777 - accuracy: 0.5448


[1.2776960134506226, 0.5447999835014343]

In [12]:
y_pred_sigmoid = code1.predict_using_sigmoid(x_test)
y_pred_sigmoid = np.argmax(y_pred_sigmoid, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_sigmoid)
print("Confusion Matrix:")
print(conf_matrix)

313/313 [==============================] - 1s 2ms/step
Confusion Matrix:
[[547  69  39  27  17   5  15  17 221  43]
 [ 13 768   7  13   6   8   6   8  59 112]
 [ 66  21 414 116 105  60  77  77  41  23]
 [ 27  30  72 443  56 149  80  67  36  40]
 [ 38  21 156  88 419  38  89 102  38  11]
 [  9  16  74 245  46 412  43 105  30  20]
 [  9  26  80  91  81  25 618  25  24  21]
 [ 23  17  34  82  53  70   8 652  17  44]
 [ 51  98   8  18   9   4   5  13 745  49]
 [ 14 254   9  27   2  10  11  32  99 542]]


In [14]:
class_report = classification_report(y_test, y_pred_sigmoid)
print("\nClassification Report:")
print(class_report)


Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.55      0.61      1000
           1       0.58      0.77      0.66      1000
           2       0.46      0.41      0.44      1000
           3       0.39      0.44      0.41      1000
           4       0.53      0.42      0.47      1000
           5       0.53      0.41      0.46      1000
           6       0.65      0.62      0.63      1000
           7       0.59      0.65      0.62      1000
           8       0.57      0.74      0.65      1000
           9       0.60      0.54      0.57      1000

    accuracy                           0.56     10000
   macro avg       0.56      0.56      0.55     10000
weighted avg       0.56      0.56      0.55     10000



# Part 2 (Using ReLu)

In [15]:
class using_ReLu():
  def __init__(self):
    cnn2=models.Sequential()
    self.cnn2=cnn2
  def tarin_using_ReLu(self,x_train,y_train,epochs=15):
    self.cnn2.add(layers.Conv2D(32, (3,3),activation='relu',input_shape=(32,32,3)))
    self.cnn2.add(layers.MaxPooling2D((2, 2)))
    self.cnn2.add(layers.Conv2D(64, (3, 3), activation='relu'))
    self.cnn2.add(layers.MaxPooling2D((2, 2)))


    self.cnn2.add(layers.Flatten())
    self.cnn2.add(layers.Dense(64, activation='relu'))
    self.cnn2.add(layers.Dense(10,activation='softmax'))

    #Compile our models
    self.cnn2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    #fit data to our models
    return self.cnn2.fit(x_train, y_train, epochs=15)
  def test_cnn_using_ReLu(self, x_test,y_test):
    return self.cnn2.evaluate(x_test,y_test)
  def predict_using_relu(self,x_test):
    return self.cnn2.predict(x_test)

In [16]:
code2=using_ReLu()
code2.tarin_using_ReLu(x_train,y_train)

Epoch 1/15
938/938 [==============================] - 6s 4ms/step - loss: 1.6074 - accuracy: 0.4172
Epoch 2/15
938/938 [==============================] - 4s 4ms/step - loss: 1.2668 - accuracy: 0.5486
Epoch 3/15
938/938 [==============================] - 3s 4ms/step - loss: 1.1205 - accuracy: 0.6038
Epoch 4/15
938/938 [==============================] - 4s 4ms/step - loss: 1.0280 - accuracy: 0.6388
Epoch 5/15
938/938 [==============================] - 3s 4ms/step - loss: 0.9563 - accuracy: 0.6614
Epoch 6/15
938/938 [==============================] - 3s 4ms/step - loss: 0.8986 - accuracy: 0.6871
Epoch 7/15
938/938 [==============================] - 4s 4ms/step - loss: 0.8454 - accuracy: 0.7057
Epoch 8/15
938/938 [==============================] - 3s 4ms/step - loss: 0.7968 - accuracy: 0.7202
Epoch 9/15
938/938 [==============================] - 3s 4ms/step - loss: 0.7524 - accuracy: 0.7364
Epoch 10/15
938/938 [==============================] - 4s 4ms/step - loss: 0.7120 - accuracy: 0.7509

In [17]:
code2.test_cnn_using_ReLu(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.1413 - accuracy: 0.6515


[1.1412692070007324, 0.6514999866485596]

In [18]:
code2.test_cnn_using_ReLu(x_add_test,y_add_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.2901 - accuracy: 0.6037


[1.2901009321212769, 0.6036999821662903]

In [19]:
y_pred_relu = code2.predict_using_relu(x_test)
y_pred_relu = np.argmax(y_pred_relu, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_relu)
print("Confusion Matrix:")
print(conf_matrix)

313/313 [==============================] - 1s 2ms/step
Confusion Matrix:
[[637  10  56  29  31   7   7  35 136  52]
 [ 27 755   7  16  11   6   7  18  39 114]
 [ 60   6 483  93 164  54  32  70  21  17]
 [ 15  13  76 479 116 132  45  87  22  15]
 [ 12   3  62  53 667  26  30 125  14   8]
 [ 10   4  73 207  67 485  16 114  15   9]
 [  8  10  53  92 127  22 641  26  10  11]
 [  5   3  25  36  63  28   3 821   5  11]
 [ 50  35  16  16  12   5   5  10 811  40]
 [ 34  84   9  30  13   6   8  35  45 736]]


In [20]:
class_report = classification_report(y_test, y_pred_relu)
print("\nClassification Report:")
print(class_report)


Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.64      0.69      1000
           1       0.82      0.76      0.79      1000
           2       0.56      0.48      0.52      1000
           3       0.46      0.48      0.47      1000
           4       0.52      0.67      0.59      1000
           5       0.63      0.48      0.55      1000
           6       0.81      0.64      0.71      1000
           7       0.61      0.82      0.70      1000
           8       0.73      0.81      0.77      1000
           9       0.73      0.74      0.73      1000

    accuracy                           0.65     10000
   macro avg       0.66      0.65      0.65     10000
weighted avg       0.66      0.65      0.65     10000

